In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
import os
import json 

In [13]:
# 2019년 12달의 모든 aws데이터를 하나의 dataframe으로 순서대로 합치기
import pandas as pd
import glob

# csv_files = sorted(glob('/raid/workspace/honghee/data_weather/train_x/aws/*/*.csv'))
path = r'/raid/workspace/honghee/data_weather/train_x/aws/2019' 
all_files = sorted(glob.glob(path + "/*.csv"))
# print(all_files)
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

aws = pd.concat(li, axis=0, ignore_index=True)
aws

['/raid/workspace/honghee/data_weather/train_x/aws/2019/AWS_201901.csv', '/raid/workspace/honghee/data_weather/train_x/aws/2019/AWS_201902.csv', '/raid/workspace/honghee/data_weather/train_x/aws/2019/AWS_201903.csv', '/raid/workspace/honghee/data_weather/train_x/aws/2019/AWS_201904.csv', '/raid/workspace/honghee/data_weather/train_x/aws/2019/AWS_201905.csv', '/raid/workspace/honghee/data_weather/train_x/aws/2019/AWS_201906.csv', '/raid/workspace/honghee/data_weather/train_x/aws/2019/AWS_201907.csv', '/raid/workspace/honghee/data_weather/train_x/aws/2019/AWS_201908.csv', '/raid/workspace/honghee/data_weather/train_x/aws/2019/AWS_201909.csv', '/raid/workspace/honghee/data_weather/train_x/aws/2019/AWS_201910.csv', '/raid/workspace/honghee/data_weather/train_x/aws/2019/AWS_201911.csv', '/raid/workspace/honghee/data_weather/train_x/aws/2019/AWS_201912.csv']


,Unnamed: 0,지점번호,시각,평균 기온,최고 기온,최저 기온,평균 풍속,합성 풍속,합성 풍향,최대 순간 풍속,최대 순간 풍속 풍향,평균 전운량,합계 강수량,평균 현지기압,평균 해면기압,이슬점온도,10분 평균 시정,운고
0,0,90,2019-01-01 01:00:00,-1.8,-1.1,-2.8,0.9,0.3,350.1,2.7,205.9,0.0,0.0,1028.9,1031.2,-23.1,20000.0,7620.0
1,1,90,2019-01-01 02:00:00,-2.2,-1.7,-3.0,1.0,0.8,327.6,3.5,341.2,0.0,0.0,1028.8,1031.1,-24.0,20000.0,7620.0
2,2,90,2019-01-01 03:00:00,-2.9,-2.1,-4.0,1.3,0.8,299.7,3.7,5.6,0.0,0.0,1028.7,1031.1,-23.6,20000.0,7620.0
3,3,90,2019-01-01 04:00:00,-3.4,-2.1,-4.3,1.1,1.0,286.3,2.9,293.3,0.0,0.0,1028.3,1030.6,-23.6,20000.0,7620.0
4,4,90,2019-01-01 05:00:00,-3.9,-3.0,-4.9,1.1,1.0,287.5,2.3,248.2,0.2,0.0,1027.8,1030.1,-23.4,20000.0,7620.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5606395,476155,989,2019-12-31 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5606396,476156,989,2019-12-31 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5606397,476157,989,2019-12-31 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5606398,476158,989,2019-12-31 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# 지점번호와 권역을 연결하고 세부권역이름과 대표 9개 권역을 매칭시키는데 필요한 region csv 만들기
# 앞의 3글자를 따서 세부권역으로 저장하고 나중에 대표 권역안에 포함되어있는지 판별해서 매칭할 예정

region_csv = pd.read_csv('/raid/workspace/honghee/data_weather/train_x/aws/region_filled.csv')
region_csv = region_csv[['지점', '지점주소', '위도', '경도', '노장해발고도(m)']][0:628]
region_csv['권역'] = region_csv['지점주소'].astype(str).str[0:3]
region_csv = region_csv.drop('지점주소', axis=1)
# 예외 처리
region_csv = region_csv.replace('제주특', '제주도')
region_csv = region_csv.replace('서귀포', '제주도')
region_csv.to_csv("/raid/workspace/honghee/data_weather/train_x/aws/region.csv", index=False)

region_csv = pd.read_csv("/raid/workspace/honghee/data_weather/train_x/aws/region.csv")
region_csv

,지점,위도,경도,노장해발고도(m),권역
0,90.0,38.2509,128.5647,17.53,강원도
1,93.0,37.9475,127.7547,95.61,강원도
2,95.0,38.1479,127.3042,155.48,강원도
3,96.0,37.2395,131.8698,99.08,경상북
4,98.0,37.9019,127.0607,115.62,경기도
...,...,...,...,...,...
623,966.0,37.1161,126.3856,38.00,인천광
624,967.0,37.1189,126.6128,70.00,인천광
625,968.0,35.0810,129.0450,78.58,부산광
626,969.0,35.0910,129.1270,37.72,부산광


In [ ]:
# label csv 전처리: 9개 권역의 권역당 13개의 label을 모두 합치고 세부권역과 매칭시키기 위해 앞의 3글자씩 떼어서 이름 바꾸기

target_csv = pd.read_csv('/raid/workspace/honghee/data_weather/train_y/weather_news/2014_2020_target.csv')
print(list(target_csv.columns))
# 각 권역의 첫 번째 col이름 바꿔 놓고 밑에서 for 문으로 하나씩 추가
target_csv["서울특인천광경기도"] = target_csv["서울·인천·경기도_강풍"].astype(str)
target_csv["부산광울산광경상남"] = target_csv["부산·울산·경상남도_강풍"].astype(str)
target_csv["대구광경상북"] = target_csv["대구·경상북도_강풍"].astype(str)
target_csv["광주광전라남"] = target_csv["광주·전라남도_강풍"].astype(str)
target_csv["전라북"] = target_csv["전라북도_강풍"].astype(str)
target_csv["대전광세종특충청남"] = target_csv["대전·세종·충청남도_강풍"].astype(str)
target_csv["충청북"] = target_csv["충청북도_강풍"].astype(str)
target_csv["강원도"] = target_csv["강원도_강풍"].astype(str)
target_csv["제주도"] = target_csv["제주도_강풍"].astype(str)


for i in range(12): # 13개의 label을 떼어서 합치기
    target_csv["서울특인천광경기도"] += target_csv.iloc[:, i+1].astype(str)
    target_csv["부산광울산광경상남"] += target_csv.iloc[:, i+14].astype(str)
    target_csv["대구광경상북"] += target_csv.iloc[:, i+27].astype(str)
    target_csv["광주광전라남"] += target_csv.iloc[:, i+40].astype(str)
    target_csv["전라북"] += target_csv.iloc[:, i+53].astype(str)
    target_csv["대전광세종특충청남"] += target_csv.iloc[:, i+66].astype(str)
    target_csv["충청북"] += target_csv.iloc[:, i+79].astype(str)
    target_csv["강원도"] += target_csv.iloc[:, i+92].astype(str)
    target_csv["제주도"] += target_csv.iloc[:, i+105].astype(str)
target_csv['서울특인천광경기도'] = target_csv.서울특인천광경기도.apply(lambda x: list(x))
target_csv['부산광울산광경상남'] = target_csv.부산광울산광경상남.apply(lambda x: list(x))
target_csv['대구광경상북'] = target_csv.대구광경상북.apply(lambda x: list(x))
target_csv['광주광전라남'] = target_csv.광주광전라남.apply(lambda x: list(x))
target_csv['전라북'] = target_csv.전라북.apply(lambda x: list(x))
target_csv['대전광세종특충청남'] = target_csv.대전광세종특충청남.apply(lambda x: list(x))
target_csv['충청북'] = target_csv.충청북.apply(lambda x: list(x))
target_csv['강원도'] = target_csv.강원도.apply(lambda x: list(x))
target_csv['제주도'] = target_csv.제주도.apply(lambda x: list(x))
target = target_csv.iloc[:, 131:140]
# print(target.iloc[1,1])
# print(target)
target.to_csv("/raid/workspace/honghee/data_weather/train_y/weather_news/target.csv", index=False)
# label = pd.read_csv("/raid/workspace/honghee/data_weather/train_y/weather_news/target.csv")
# print(label)

In [15]:
# 변환된 label csv
# label을 str으로 한 이유는 int로 했더니 저장이 이상하게 되어서 일단 str으로 하고 나중에 데이터로더 만들때 int로 바꾸어서 작업 예정
label_csv = pd.read_csv("/raid/workspace/honghee/data_weather/train_y/weather_news/target.csv")
label_csv

,서울특인천광경기도,부산광울산광경상남,대구광경상북,광주광전라남,전라북,대전광세종특충청남,충청북,강원도,제주도
0,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
1,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
2,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
3,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
4,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
...,...,...,...,...,...,...,...,...,...
52603,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '1', '0', '0', '0', '0', '0', '0', ...","['0', '0', '2', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '2', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
52604,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '1', '0', '0', '0', '0', '0', '0', ...","['0', '0', '2', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '2', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
52605,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '2', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '2', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
52606,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '2', 

In [16]:
# aws csv 데이터에 위도, 경도, 해발고도 및 지점번호와 매칭되는 해당 권역 등 4가지 col 추가

do_list = []
latitude_list = []
longtitude_list = []
elevation_list = []
for i in range(len(aws)):
    region = aws.loc[i, '지점번호']
    if region_csv.index[region_csv['지점'] == region].tolist() == []:  # 지점번호 정보가 없는 지역 존재
        # none 추가 후 나중에 모두 제거 예정
        do_list.append('none')
        latitude_list.append('none')
        longtitude_list.append('none')
        elevation_list.append('none')
    else:
        # 지점번호와 매칭되도록 4가지 col 추가
        region_index = region_csv.index[region_csv['지점'] == region].tolist()[0]
        do = region_csv.loc[region_index, '권역']
        latitude = region_csv.loc[region_index, '위도']
        longtitude = region_csv.loc[region_index, '경도']
        elevation = region_csv.loc[region_index, '노장해발고도(m)']

        do_list.append(do)
        latitude_list.append(latitude)
        longtitude_list.append(longtitude)
        elevation_list.append(elevation)
        
aws['위도']=latitude_list
aws['경도']=longtitude_list
aws['해발고도']=elevation_list
aws['권역']=do_list
# aws = aws.drop(aws[aws.권역 == 'none'].index)
# print(aws)
# aws.to_csv("/raid/workspace/honghee/data_weather/train_x/aws/2019aws.csv", index=False)


NameError: name 'sample_csv' is not defined

In [17]:
# 지점번호가 없는 즉, none인 지역은 제거
aws = aws.drop(aws[aws.권역 == 'none'].index)
print(aws)
aws.to_csv("/raid/workspace/honghee/data_weather/train_x/aws/2019aws.csv", index=False)

         Unnamed: 0  지점번호                   시각  평균 기온  최고 기온  최저 기온  평균 풍속  \
0                 0    90  2019-01-01 01:00:00   -1.8   -1.1   -2.8    0.9   
1                 1    90  2019-01-01 02:00:00   -2.2   -1.7   -3.0    1.0   
2                 2    90  2019-01-01 03:00:00   -2.9   -2.1   -4.0    1.3   
3                 3    90  2019-01-01 04:00:00   -3.4   -2.1   -4.3    1.1   
4                 4    90  2019-01-01 05:00:00   -3.9   -3.0   -4.9    1.1   
...             ...   ...                  ...    ...    ...    ...    ...   
5606395      476155   989  2019-12-31 20:00:00    NaN    NaN    NaN    NaN   
5606396      476156   989  2019-12-31 21:00:00    NaN    NaN    NaN    NaN   
5606397      476157   989  2019-12-31 22:00:00    NaN    NaN    NaN    NaN   
5606398      476158   989  2019-12-31 23:00:00    NaN    NaN    NaN    NaN   
5606399      476159   989  2020-01-01 00:00:00    NaN    NaN    NaN    NaN   

         합성 풍속  합성 풍향  최대 순간 풍속  ...  합계 강수량  평균 현지기압  평균 해면기압 

In [19]:
# aws 데이터에서 target csv의 label과 매칭되는 시각, 권역의 row에 해당 label 추가
# 시계열 데이터이기 때문에 시간 별로 순서가 정해져야 하고 해당 권역에 해당 label이 있어야 나중에 dataloader 만들 때 편함

aws2019 = pd.read_csv("/raid/workspace/honghee/data_weather/train_x/aws/2019aws.csv")
# print(aws2019)
target_csv = pd.read_csv('/raid/workspace/honghee/data_weather/train_y/weather_news/2014_2020_target.csv')
label_csv["시각"] = target_csv["datetime"]

aws2019 = aws2019.drop('Unnamed: 0', axis=1)
# aws2019 = aws2019.replace('제주특', '제주도')
# aws2019 = aws2019.replace('서귀포', '제주도')

label_list = []
label_names = list(label_csv.columns)

for i in range(len(aws2019)):
    time = aws2019.loc[i, '시각']  
    region_name = aws2019.loc[i, '권역']  

    label_x = label_csv.index[label_csv['시각'] == time].tolist()[0]
    label_y = [i for i in range(len(label_names)) if region_name in label_names[i]][0]
    label = label_csv.iloc[label_x, label_y]
    label_list.append(label)

aws2019['weather_label']=label_list
print(aws2019)
aws2019.to_csv("/raid/workspace/honghee/data_weather/train_x/aws/aws_2019.csv", index=False)


         지점번호                   시각  평균 기온  최고 기온  최저 기온  평균 풍속  합성 풍속  합성 풍향  \
0          90  2019-01-01 01:00:00   -1.8   -1.1   -2.8    0.9    0.3  350.1   
1          90  2019-01-01 02:00:00   -2.2   -1.7   -3.0    1.0    0.8  327.6   
2          90  2019-01-01 03:00:00   -2.9   -2.1   -4.0    1.3    0.8  299.7   
3          90  2019-01-01 04:00:00   -3.4   -2.1   -4.3    1.1    1.0  286.3   
4          90  2019-01-01 05:00:00   -3.9   -3.0   -4.9    1.1    1.0  287.5   
...       ...                  ...    ...    ...    ...    ...    ...    ...   
5501275   989  2019-12-31 20:00:00    NaN    NaN    NaN    NaN    NaN    NaN   
5501276   989  2019-12-31 21:00:00    NaN    NaN    NaN    NaN    NaN    NaN   
5501277   989  2019-12-31 22:00:00    NaN    NaN    NaN    NaN    NaN    NaN   
5501278   989  2019-12-31 23:00:00    NaN    NaN    NaN    NaN    NaN    NaN   
5501279   989  2020-01-01 00:00:00    NaN    NaN    NaN    NaN    NaN    NaN   

         최대 순간 풍속  최대 순간 풍속 풍향  ...  평균

In [27]:
# 결측값 전처리
# 같은 지점끼리 모은 후 결측값 linear interpolation 처리 

aws_2019 = pd.read_csv("/raid/workspace/honghee/data_weather/train_x/aws/aws_2019.csv")
regionlist_csv = pd.read_csv('/raid/workspace/honghee/data_weather/train_x/aws/region_filled.csv')
regionlist = regionlist_csv['지점'][0:628].tolist()
for r in regionlist:  # 같은 지점끼리 모아서 저장
    y = aws_2019.loc[aws_2019['지점번호'] == r]
    globals()[f'aws_{int(r)}'] = y # aws_지점번호 인 csv 생성 및 변수명 할당
    # 같은 지점끼리 linear interpolation, 기간 내내 nan값인 경우 변화 없음
    globals()[f'aws_{int(r)}'] = globals()[f'aws_{int(r)}'].interpolate()  

aws19 = pd.DataFrame()
for r in regionlist:  # 같은 지점끼리 모으기
    aws19 = pd.concat([aws19, globals()[f'aws_{int(r)}']])
aws19 = aws19.fillna(0) # 기간 내내 nan 값인 경우 0으로 바꾸어 학습에 영향 미치지 않도록 전처리
aws19

,지점번호,시각,평균 기온,최고 기온,최저 기온,평균 풍속,합성 풍속,합성 풍향,최대 순간 풍속,최대 순간 풍속 풍향,...,평균 현지기압,평균 해면기압,이슬점온도,10분 평균 시정,운고,위도,경도,해발고도,권역,weather_label
0,90,2019-01-01 01:00:00,-1.8,-1.1,-2.8,0.9,0.3,350.1,2.7,205.9,...,1028.9,1031.2,-23.1,20000.0,7620.0,38.2509,128.5647,17.53,강원도,"['0', '0', '0', '0', '0', '0', '0', '1', '0', ..."
1,90,2019-01-01 02:00:00,-2.2,-1.7,-3.0,1.0,0.8,327.6,3.5,341.2,...,1028.8,1031.1,-24.0,20000.0,7620.0,38.2509,128.5647,17.53,강원도,"['0', '0', '0', '0', '0', '0', '0', '1', '0', ..."
2,90,2019-01-01 03:00:00,-2.9,-2.1,-4.0,1.3,0.8,299.7,3.7,5.6,...,1028.7,1031.1,-23.6,20000.0,7620.0,38.2509,128.5647,17.53,강원도,"['0', '0', '0', '0', '0', '0', '0', '1', '0', ..."
3,90,2019-01-01 04:00:00,-3.4,-2.1,-4.3,1.1,1.0,286.3,2.9,293.3,...,1028.3,1030.6,-23.6,20000.0,7620.0,38.2509,128.5647,17.53,강원도,"['0', '0', '0', '0', '0', '0', '0', '1', '0', ..."
4,90,2019-01-01 05:00:00,-3.9,-3.0,-4.9,1.1,1.0,287.5,2.3,248.2,...,1027.8,1030.1,-23.4,20000.0,7620.0,38.2509,128.5647,17.53,강원도,"['0', '0', '0', '0', '0', '0', '0', '1', '0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5501275,989,2019-12-31 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,33.2579,126.3299,90.00,제주도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
5501276,989,2019-12-31 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,33.2579,126.3299,90.00,제주도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
5501277,989,2019-12-31 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,33.2579,126.3299,90.00,제주도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
5501278,989,2019-12-31 23:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,33.2579,126.3299,90.00,제주도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."


In [28]:
# normalization: min-max scaling
# 분석에 사용할 feature 선택
csv_features = ['평균 기온', '최고 기온', '최저 기온', '평균 풍속', '합성 풍속', '합성 풍향', '최대 순간 풍속', '최대 순간 풍속 풍향', 
                '평균 전운량', '합계 강수량', '평균 현지기압', '평균 해면기압', '이슬점온도', '10분 평균 시정', '운고', '위도', '경도',
               '해발고도']

temp_csv = aws19[csv_features]
max_arr, min_arr = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()

# feature 별 최대값, 최솟값 dictionary 생성
csv_feature_dict = {csv_features[i]:[min_arr[i], max_arr[i]] for i in range(len(csv_features))}

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# minmax scaling으로 normalization 처리
aws_norm = pd.DataFrame(scaler.fit_transform(temp_csv), columns=temp_csv.columns)
aws19 = aws19.drop(csv_features, axis=1)
aws_final = pd.concat([aws19, aws_norm], axis=1)
aws_final.to_csv("/raid/workspace/honghee/data_weather/train_x/aws/aws_final_2019.csv", index=False)

In [29]:
# 전처리 완료된 2019 aws csv 데이터
final = pd.read_csv("/raid/workspace/honghee/data_weather/train_x/aws/aws_final_2019.csv")
final

,지점번호,시각,권역,weather_label,평균 기온,최고 기온,최저 기온,평균 풍속,합성 풍속,합성 풍향,...,평균 전운량,합계 강수량,평균 현지기압,평균 해면기압,이슬점온도,10분 평균 시정,운고,위도,경도,해발고도
0,90,2019-01-01 01:00:00,강원도,"['0', '0', '0', '0', '0', '0', '0', '1', '0', ...",0.316062,0.319728,0.830944,0.023316,0.0003,0.972500,...,0.916667,0.0,0.989898,0.988592,0.220930,0.322799,0.57771,0.992434,0.965860,0.009870
1,90,2019-01-01 02:00:00,강원도,"['0', '0', '0', '0', '0', '0', '0', '1', '0', ...",0.309154,0.309524,0.830101,0.025907,0.0008,0.910000,...,0.916667,0.0,0.989802,0.988496,0.207849,0.322799,0.57771,0.992434,0.965860,0.009870
2,90,2019-01-01 03:00:00,강원도,"['0', '0', '0', '0', '0', '0', '0', '1', '0', ...",0.297064,0.302721,0.825885,0.033679,0.0008,0.832500,...,0.916667,0.0,0.989706,0.988496,0.213663,0.322799,0.57771,0.992434,0.965860,0.009870
3,90,2019-01-01 04:00:00,강원도,"['0', '0', '0', '0', '0', '0', '0', '1', '0', ...",0.288428,0.302721,0.824621,0.028497,0.0010,0.795278,...,0.916667,0.0,0.989321,0.988016,0.213663,0.322799,0.57771,0.992434,0.965860,0.009870
4,90,2019-01-01 05:00:00,강원도,"['0', '0', '0', '0', '0', '0', '0', '1', '0', ...",0.279793,0.287415,0.822091,0.028497,0.0010,0.798611,...,0.918519,0.0,0.988840,0.987537,0.216570,0.322799,0.57771,0.992434,0.965860,0.009870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5501275,989,2019-12-31 20:00:00,제주도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...",0.347150,0.338435,0.842749,0.000000,0.0000,0.000000,...,0.916667,0.0,0.000000,0.000000,0.556686,0.000000,0.00000,0.862889,0.942775,0.053406
5501276,989,2019-12-31 21:00:00,제주도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...",0.347150,0.338435,0.842749,0.000000,0.0000,0.000000,...,0.916667,0.0,0.000000,0.000000,0.556686,0.000000,0.00000,0.862889,0.942775,0.053406
5501277,989,2019-12-31 22:00:00,제주도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...",0.347150,0.338435,0.842749,0.000000,0.0000,0.000000,...,0.916667,0.0,0.000000,0.000000,0.556686,0.000000,0.00000,0.862889,0.942775,0.053406
5501278,989,2019-12-31 23:00:00,제주도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...",0.347150,0.338435,0.842749,0.000000,0.0000,0.000000,...,0.916667,0.0,0.000000,0.000000,0.556686,0.000000,0.00000,0.862889,0.942775,0.053406
